# **Deep transfer learning based on fine-tuning**
---
<font color=black size=3 face=雅黑>**Envirionment**: Tensorflow = 1.14.0 Keras = 2.2.4</font>

<font color=black size=3 face=雅黑>**A schematic introduction of implementation proposed parameter-based models**</font>

<font color=black size=2 face=雅黑>In what follows, we explain how to implement a fine-tune (parameter-based model) in Keras for DTL-IFD

Schematically, the designed parameter-based DTL looks like this:

* A `Base_Net` model based on 1D_CNN (2D_CNN) maps vectors of shape `1024x1(64x64x3)` to extracted feature of shape `(128,1)`.
    
* A `Finetune_Net` model maps CNN feature `(128,1)` to a Probability output with ten categories`(10,)`.
    
*Step 1* : We first use the source data to train `Base_Net` and `Finetune_Net`, making sure that these two models could correctly classify 10 categories of source data

*Step 2* : We **freeze** the trained model parameters of  `Base_Net`, use this parameter for target data

*Step 3* : We import the target data into the model and only optimize the parameters of `Finetune_Net`.

**By iteratively conducting step 1 to step 3, the domain shift could be covered and the classfication task from source domain to target domain could be implemented** </font>

# Import the basic modules

In [1]:
import numpy as np
from keras.layers import Input, Dense, Activation,Reshape, Flatten, Conv2D,Conv1D,BatchNormalization, MaxPooling2D,PReLU, Dropout,Conv1D,GlobalAveragePooling1D
from keras import models
from keras.models import load_model
from keras.models import Model
from keras.optimizers import SGD
from keras.utils import to_categorical
from sklearn.datasets import make_blobs
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.layers.advanced_activations import LeakyReLU

Using TensorFlow backend.
C:\Users\phoen\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\phoen\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\phoen\Anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\phoen\Anaconda3\envs\te

# Load Data

In [2]:
# Load 1D data
X_0_1D = np.load("./DE/X_0.npy").reshape((-1,1024,1))
X_1_1D = np.load("./DE/X_1.npy").reshape((-1,1024,1))
X_2_1D = np.load("./DE/X_2.npy").reshape((-1,1024,1))
X_3_1D = np.load("./DE/X_3.npy").reshape((-1,1024,1))
# Load 2D data
X_0_2D = np.load("./DE/Case1_FFT.npy").reshape((-1,64,64,3))/255
X_1_2D = np.load("./DE/Case2_FFT.npy").reshape((-1,64,64,3))/255
X_2_2D = np.load("./DE/Case3_FFT.npy").reshape((-1,64,64,3))/255
X_3_2D = np.load("./DE/Case4_FFT.npy").reshape((-1,64,64,3))/255
Y_0 = np.load("./DE/Y_0.npy")
Y_1 = np.load("./DE/Y_1.npy")
Y_2 = np.load("./DE/Y_2.npy")
Y_3 = np.load("./DE/Y_3.npy")
def Shuffle(x,y,split):
    index=np.random.choice(a=len(x), size=len(x), replace=False, p=None)
    X=[]
    Y=[]
    for i in range(len(x)):
        slice_x = x[index[i]]
        slice_y = y[index[i]]
        X.append(slice_x)
        Y.append(slice_y)
    X = np.array(X)
    Y = np.array(Y)
    Y = to_categorical(Y)
    train_X = X[0:int(len(x)*split)]
    train_Y = Y[0:int(len(x)*split)]
    test_X = X[int(len(x)*split):]
    test_Y = Y[int(len(x)*split):]
    return train_X,train_Y,test_X,test_Y

# Defination of feature-based model

In [3]:
# defination of Base_Net for 1D-data
def CNN_1D():
    """Creates three different models, one used for source only training, two used for domain adaptation"""
    inputs = Input(shape=(1024,1)) 
    x4 = Conv1D(128,3)(inputs)
    x4 = LeakyReLU()(x4)
    x4 = Dropout(0.25)(x4)
    x4 = Conv1D(64,3)(x4)
    x4 = LeakyReLU()(x4)
    x4 = Dropout(0.25)(x4)
    x4 = Conv1D(32,3)(x4)
    x4 = LeakyReLU()(x4)
    x4 = Dropout(0.25)(x4)
    x4 = Flatten()(x4)
    x4 = Dense(128)(x4)
    x4 = LeakyReLU()(x4)
    x4 = Reshape((128,1))(x4)
    model = Model(inputs=inputs, outputs=x4)
    return model
# defination of Base_Net for 2D-data
def CNN_2D():
    """Creates three different models, one used for source only training, two used for domain adaptation"""
    inputs = Input(shape=(64,64,3)) 
    x4 = Conv2D(128,(3,3))(inputs)
    x4 = LeakyReLU()(x4)
    x4 = Dropout(0.25)(x4)
    x4 = Conv2D(64,(3,3))(x4)
    x4 = LeakyReLU()(x4)
    x4 = Dropout(0.25)(x4)
    x4 = Conv2D(32,(3,3))(x4)
    x4 = LeakyReLU()(x4)
    x4 = Dropout(0.25)(x4)
    x4 = Flatten()(x4)
    x4 = Dense(128)(x4)
    x4 = LeakyReLU()(x4)
    x4 = Reshape((128,1))(x4)
    model = Model(inputs=inputs, outputs=x4)
    return model
# defination of Finetune_net
def Classify():
    inputs = Input(shape=(128,1)) 
    x4= Conv1D(32,3, name="mo1")(inputs)
    x4 = LeakyReLU(name="mo2")(x4)
    x4 = Dropout(0.25)(x4)
    x4 = Conv1D(16,3, name="mo3")(x4)
    x4 = LeakyReLU(name="mo4")(x4)
    x4 = Flatten()(x4)
    x4 = Dense(64,name="mo5")(x4)
    x4 = LeakyReLU(name="mo6")(x4)
    x4 = Dense(10, activation='softmax', name="mo")(x4)  
    model = Model(inputs=inputs, outputs=x4)
    return model

In [4]:
base_net_1D = CNN_1D()
base_net_2D = CNN_2D()
finetune_net = Classify()

model_1D = models.Sequential()
model_1D.add(base_net_1D)
model_1D.add(finetune_net)
model_1D.compile(optimizer="Adam",loss='categorical_crossentropy',metrics=['accuracy'])

model_2D = models.Sequential()
model_2D.add(base_net_2D)
model_2D.add(finetune_net)
model_2D.compile(optimizer="Adam",loss='categorical_crossentropy',metrics=['accuracy'])





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




# Pre-training

In [5]:
trainX,trainY,testX,testY = Shuffle(X_0_1D,Y_0,split=0.8)
model_1D.fit(trainX,trainY,batch_size=64,epochs=50,validation_data=[testX,testY])
model_1D.save_weights("pre-trained-1D.h5")
trainX,trainY,testX,testY = Shuffle(X_0_2D,Y_0,split=0.8)
model_2D.fit(trainX,trainY,batch_size=64,epochs=50,validation_data=[testX,testY])
model_2D.save_weights("pre-trained-2D.h5")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 8000 samples, validate on 2000 samples
Epoch 1/50
8000/8000 [==============================] - 8s 988us/step - loss: 0.8319 - acc: 0.6749 - val_loss: 0.2585 - val_acc: 0.9250
Epoch 2/50
8000/8000 [==============================] - 2s 276us/step - loss: 0.2219 - acc: 0.9289 - val_loss: 0.1567 - val_acc: 0.9370
Epoch 3/50
8000/8000 [==============================] - 2s 277us/step - loss: 0.1138 - acc: 0.9584 - val_loss: 0.0599 - val_acc: 0.9790
Epoch 4/50
8000/8000 [==============================] - 2s 275us/step - loss: 0.0376 - acc: 0.9873 - val_loss: 0.2556 - val_acc: 0.9235
Epoch 5/50
8000/8000 [==============================] - 2s 276us/step - loss: 0.0113 - acc: 0.9964 - val_loss: 0.0740 - val_acc: 0.9720
Epoch 6/50
8000/8000 [==============================] - 2s 276us/step - loss: 0.0442 - acc: 0.9850 - val_loss: 0.0706 - val_acc: 0.9725
Epoch 7/50
8000/8000 [====================

8000/8000 [==============================] - 8s 1ms/step - loss: 14.5426 - acc: 0.0977 - val_loss: 14.3612 - val_acc: 0.1090
Epoch 7/50
8000/8000 [==============================] - 8s 1ms/step - loss: 14.5426 - acc: 0.0977 - val_loss: 14.3612 - val_acc: 0.1090
Epoch 8/50
8000/8000 [==============================] - 8s 1ms/step - loss: 14.5426 - acc: 0.0977 - val_loss: 14.3612 - val_acc: 0.1090
Epoch 9/50
8000/8000 [==============================] - 8s 1ms/step - loss: 14.5426 - acc: 0.0977 - val_loss: 14.3612 - val_acc: 0.1090
Epoch 10/50
8000/8000 [==============================] - 8s 1ms/step - loss: 14.5426 - acc: 0.0977 - val_loss: 14.3612 - val_acc: 0.1090
Epoch 11/50
8000/8000 [==============================] - 8s 1ms/step - loss: 14.5426 - acc: 0.0977 - val_loss: 14.3612 - val_acc: 0.1090
Epoch 12/50
8000/8000 [==============================] - 8s 1ms/step - loss: 14.5426 - acc: 0.0977 - val_loss: 14.3612 - val_acc: 0.1090
Epoch 13/50
8000/8000 [==============================] -

# Fine tune

In [6]:
model_1D.load_weights('pre-trained-1D.h5', by_name=True)
base_net_1D.trainable=False
model_2D.load_weights('pre-trained-2D.h5', by_name=True)
base_net_2D.trainable=False

In [7]:
trainDX,trainDY,testDX,testDY = Shuffle(X_1_1D,Y_1,split=0.8)
model_1D.fit(trainDX,trainDY,batch_size=64,epochs=5,validation_data=[testDX,testDY])
trainDX,trainDY,testDX,testDY = Shuffle(X_1_2D,Y_1,split=0.8)
model_2D.fit(trainDX,trainDY,batch_size=16,epochs=5,validation_data=[testDX,testDY])

Train on 8000 samples, validate on 2000 samples
Epoch 1/5
 576/8000 [=>............................] - ETA: 1s - loss: 3.0328 - acc: 0.5972

C:\Users\phoen\Anaconda3\envs\tensorflow\lib\site-packages\keras\engine\training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


8000/8000 [==============================] - 2s 272us/step - loss: 0.4791 - acc: 0.8830 - val_loss: 0.0431 - val_acc: 0.9865
Epoch 2/5
8000/8000 [==============================] - 2s 273us/step - loss: 0.0345 - acc: 0.9903 - val_loss: 0.0622 - val_acc: 0.9780
Epoch 3/5
8000/8000 [==============================] - 2s 270us/step - loss: 0.0220 - acc: 0.9929 - val_loss: 0.0265 - val_acc: 0.9895
Epoch 4/5
8000/8000 [==============================] - 5s 658us/step - loss: 0.0051 - acc: 0.9988 - val_loss: 0.0278 - val_acc: 0.9895
Epoch 5/5
8000/8000 [==============================] - 6s 699us/step - loss: 0.0019 - acc: 0.9996 - val_loss: 0.0448 - val_acc: 0.9855
Train on 8000 samples, validate on 2000 samples
Epoch 1/5
8000/8000 [==============================] - 27s 3ms/step - loss: 14.5023 - acc: 0.1003 - val_loss: 14.5224 - val_acc: 0.0990
Epoch 2/5
8000/8000 [==============================] - 26s 3ms/step - loss: 14.5023 - acc: 0.1003 - val_loss: 14.5224 - val_acc: 0.0990
Epoch 3/5
8000/

In [8]:
model_1D.save("Pretrain-Task1_1D.h5")
model_2D.save("Pretrain-Task1_2D.h5")

# Model evaluation

In [9]:
model=load_model("Pretrain-Task1_1D.h5")
trainDX,trainDY,testDX,testDY = Shuffle(X_1_1D,Y_1,split=0.8)
def test_model(model,testDX,testDY):
    acc=np.zeros(10,)
    for i in range(10):
        acc[i]=model.evaluate(testDX[0+(len(testDX)//10)*i:(len(testDX)//10)+(len(testDX)//10)*i],testDY[0+(len(testDX)//10)*i:(len(testDX)//10)+(len(testDX)//10)*i])[1]
    print("Parameter-based model on 1D data: Mean acc: %.2f"%(np.mean(acc)*100)+"%","Prediction variance: %.2f"%(np.var(acc)*100)+"%")

test_model(model,testDX,testDY)

model=load_model("Pretrain-Task1_2D.h5")
trainDX,trainDY,testDX,testDY = Shuffle(X_1_2D,Y_1,split=0.8)
def test_model(model,testDX,testDY):
    acc=np.zeros(10,)
    for i in range(10):
        acc[i]=model.evaluate(testDX[0+(len(testDX)//10)*i:(len(testDX)//10)+(len(testDX)//10)*i],testDY[0+(len(testDX)//10)*i:(len(testDX)//10)+(len(testDX)//10)*i])[1]
    print("Parameter-based model on 2D data: Mean acc: %.2f"%(np.mean(acc)*100)+"%","Prediction variance: %.2f"%(np.var(acc)*100)+"%")

test_model(model,testDX,testDY)

200/200 [==============================] - 0s 115us/step
Parameter-based model on 1D data: Mean acc: 99.55% Prediction variance: 0.00%
200/200 [==============================] - 0s 888us/step
Parameter-based model on 2D data: Mean acc: 9.20% Prediction variance: 0.06%
